In [ ]:
from plotly.offline import init_notebook_mode, iplot
from wordcloud import WordCloud
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import plotly.plotly as py
from plotly import tools
from datetime import date
import pandas as pd
import numpy as np 
import seaborn as sns
import random 
import warnings
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

path = "../input/"

def bar_hor(df, col, title, color, w=None, h=None, lm=0, limit=100, return_trace=False, rev=False, xlb = False):
    cnt_srs = df[col].value_counts()
    yy = cnt_srs.head(limit).index[::-1] 
    xx = cnt_srs.head(limit).values[::-1] 
    if rev:
        yy = cnt_srs.tail(limit).index[::-1] 
        xx = cnt_srs.tail(limit).values[::-1] 
    if xlb:
        trace = go.Bar(y=xlb, x=xx, orientation = 'h', marker=dict(color=color))
    else:
        trace = go.Bar(y=yy, x=xx, orientation = 'h', marker=dict(color=color))
    if return_trace:
        return trace 
    layout = dict(title=title, margin=dict(l=lm), width=w, height=h)
    data = [trace]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

def bar_hor_noagg(x, y, title, color, w=None, h=None, lm=0, limit=100, rt=False):
    trace = go.Bar(y=x, x=y, orientation = 'h', marker=dict(color=color))
    if rt:
        return trace
    layout = dict(title=title, margin=dict(l=lm), width=w, height=h)
    data = [trace]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)


def bar_ver_noagg(x, y, title, color, w=None, h=None, lm=0, rt = False):
    trace = go.Bar(y=y, x=x, marker=dict(color=color))
    if rt:
        return trace
    layout = dict(title=title, margin=dict(l=lm), width=w, height=h)
    data = [trace]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
    
def gp(col, title):
    df1 = app_train[app_train["TARGET"] == 1]
    df0 = app_train[app_train["TARGET"] == 0]
    a1 = df1[col].value_counts()
    b1 = df0[col].value_counts()
    
    total = dict(app_train[col].value_counts())
    x0 = a1.index
    x1 = b1.index
    
    y0 = [float(x)*100 / total[x0[i]] for i,x in enumerate(a1.values)]
    y1 = [float(x)*100 / total[x1[i]] for i,x in enumerate(b1.values)]

    trace1 = go.Bar(x=a1.index, y=y0, name='Target : 1', marker=dict(color="#96D38C"))
    trace2 = go.Bar(x=b1.index, y=y1, name='Target : 0', marker=dict(color="#FEBFB3"))
    return trace1, trace2 

Цель данной задачи состоит в том, чтобы использовать исторические данные заявок на получение кредита для прогнозирования того, сможет ли заявитель погасить кредит.
Является задачей классификации.
Выделить классы TARGET (двоичная переменная):
- «0», погасит кредит вовремя;
- «1», будет иметь трудности с погашением;


Есть 7 различных источников данных:
- application_train/application_test: Основная обучающая и тестовая выборка с информацией о каждой кредитной заявке. Каждая строка отвечает за свой собственный кредит, идентифицируется полем SK_ID_CURR, и целевым показателем TARGR.
- bureau: Данные о предыдущих кредитах клиента в других финансовых учреждениях.
- bureau_balance: Ежемесячные данные о предыдущих кредитах в бюро.
- previous_application: Предыдущие заявки на получение кредитов в Home Credit от клиентов. 
- POS_CASH_BALANCE: Ежемесячные данные о предыдущих точках продаж или денежных ссудах, которые клиенты имели с помощью Home Credit.
- credit_card_balance: Ежемесячные данные о балансе кредитной карты.
- installments_payment: История платежей по предыдущим кредитам.


In [ ]:
app_train = pd.read_csv('../input/application_train.csv')
app_train.head()

> Всего существует 307 511 строк, которые содержат информацию о кредитах, и есть 122 переменных. 

> Целевая переменная определяет, были ли у клиента трудности с оплатой, что означает, что он просрочил платеж более чем на X дней по крайней мере по одному из первых платежей кредита. Такой случай помечается как 1 а остальные все остальные случаи как 0.

In [ ]:
# Target Variable Distribution 
bar_hor(app_train, "TARGET", "Распределение целевой переменной" , ["#44ff54", '#ff4444'], h=350, w=600, lm=200, xlb = ['Target : 1','Target : 0'])

> Целевая переменная имеет дисбаланс, большинство кредитов имеет целевое значение равное 0, что указывает на то, что у физических лиц не было никаких проблем с выплатой кредита в данный момент времени. Это около 91% кредитов, что равно примерно 282K, в то же время только 9% от общего количества кредитов (около 24K ) в этом наборе данных были связаны с проблемами с погашением кредита. 

In [ ]:
tr0 = bar_hor(app_train, "CODE_GENDER", "Distribution of CODE_GENDER Variable" ,"#f975ae", w=700, lm=100, return_trace= True)
tr1, tr2 = gp('CODE_GENDER', 'Distribution of Target with Applicant Gender')

fig = tools.make_subplots(rows=1, cols=3, print_grid=False, subplot_titles = ["Гендерное распределение" , "Пол, TARGET=1" ,"Пол, TARGET=0"])
fig.append_trace(tr0, 1, 1);
fig.append_trace(tr1, 1, 2);
fig.append_trace(tr2, 1, 3);
fig['layout'].update(height=350, showlegend=False, margin=dict(l=50));
iplot(fig);

> Женщины подали заявки на получение большей части кредитов, что почти вдвое больше, чем у мужчин. В общей сложности женщины подали около 202 448 заявок на получение займов, в то время как мужчины подали около 105 059 заявок. Однако более высокий процент (около 10% от общего числа) мужчин имел проблемы с выплатой кредита или внесением рассрочки в срок по сравнению с женщинами-заявителями (около 7%). 

In [ ]:
tr0 = bar_hor(app_train, "NAME_FAMILY_STATUS", "Distribution of CODE_GENDER Variable" ,"#f975ae", w=700, lm=100, return_trace= True)
tr1, tr2 = gp('NAME_FAMILY_STATUS', 'Distribution of Target with Applicant Gender')

fig = tools.make_subplots(rows=1, cols=3, print_grid=False, subplot_titles = ["Семейный статус" , "Статус, Target = 1" ,"Статус, Target = 0"])
fig.append_trace(tr0, 1, 1);
fig.append_trace(tr1, 1, 2);
fig.append_trace(tr2, 1, 3);
fig['layout'].update(height=350, showlegend=False, margin=dict(l=120));
iplot(fig);

> Женатые люди подали большее количество кредитных заявок около 196K, однако люди состоящие в гражданском браке имеют самый высокий процент (около 10%) кредитных проблем. 

In [ ]:
## real estate 
t = app_train['FLAG_OWN_REALTY'].value_counts()
labels = t.index
values = t.values
colors = ['#96D38C','#FEBFB3']
trace = go.Pie(labels=labels, values=values,
               hoverinfo='', textinfo='',
               textfont=dict(size=12),
               marker=dict(colors=colors,
                           line=dict(color='#fff', width=2)))
layout = go.Layout(title=' Заявители Владеющие Недвижимостью', height=400)
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)


t = app_train['FLAG_OWN_CAR'].value_counts()
labels = t.index
values = t.values
colors = ['#FEBFB3','#96D38C']
trace = go.Pie(labels=labels, values=values,
               hoverinfo='', textinfo='',
               textfont=dict(size=12),
               marker=dict(colors=colors,
                           line=dict(color='#fff', width=2)))
layout = go.Layout(title='Заявители Владеющие Автомобилем', height=400)
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

> Около 70% заявителей владеют недвижимостью, в то же время только 34% заявителей владеют автомобилем. Однако трудности с оплатой, наблюдались у заявителей которые не владели автомобилем или недвижимостью. 

In [ ]:
tr0 = bar_hor(app_train, "NAME_TYPE_SUITE", "Distribution of CODE_GENDER Variable" ,"#f975ae", w=700, lm=100, return_trace= True)
tr1 = bar_hor(app_train, "NAME_INCOME_TYPE", "Distribution of CODE_GENDER Variable" ,"#f975ae", w=700, lm=100, return_trace= True)

fig = tools.make_subplots(rows=1, cols=2, print_grid=False, subplot_titles = ['Тип апартаментов заявителя' , 'Тип дохода заявителя'])
fig.append_trace(tr0, 1, 1);
fig.append_trace(tr1, 1, 2);
fig['layout'].update(height=400, showlegend=False, margin=dict(l=100));
iplot(fig);

> Топ-3 типа апартаментов:  
    - Unaccompanined (about 248K applicants) 
    - Family (about 40K applicants)  
    - Spouse, partner (about 11K applicants)    
> Самые популярные типы доходов: 
    - Working Class (158K)
    - Commercial Associate (71K)
    - Pensiner (55K)

In [ ]:
tr1, tr2 = gp('NAME_TYPE_SUITE', 'Applicants Type Suites which repayed the loan')
fig = tools.make_subplots(rows=1, cols=2, print_grid=False, 
                          subplot_titles = ["Распределение типов когда Target = 1", "Распределение типов когда Target = 0"])
fig.append_trace(tr1, 1, 1);
fig.append_trace(tr2, 1, 2);
fig['layout'].update(height=350, showlegend=False, margin=dict(l=120));
iplot(fig);


tr1, tr2 = gp('NAME_INCOME_TYPE', 'Applicants Income Types which repayed the loan')
fig = tools.make_subplots(rows=1, cols=2, print_grid=False, 
                          subplot_titles = ["Распределение типов когда Target = 1", "Распределение типов когда Target = 0"])
fig.append_trace(tr1, 1, 1);
fig.append_trace(tr2, 1, 2);
fig['layout'].update(height=350, showlegend=False, margin=dict(l=120));
iplot(fig);

> Мы видим, что заявители, имеющие виды дохода : декретные отпуска и безработные, имеют самый высокий процент (около 40% и около 36%) целевого показателя = 1 т. е. имеют больше проблем с оплатой, в то время как пенсионеры имеют меньше всего (около 5,3%). 

In [ ]:
t = app_train['NAME_CONTRACT_TYPE'].value_counts()
labels = t.index
values = t.values
colors = ['#FEBFB3','#96D38C']
trace = go.Pie(labels=labels, values=values,
               hoverinfo='', textinfo='',
               textfont=dict(size=12),
               marker=dict(colors=colors,
                           line=dict(color='#fff', width=2)))
layout = go.Layout(title='Applicants Contract Type', height=400)
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

> Денежные займы составляют большую часть от общего числа кредитов. Возобновляемые кредиты имеют значительно меньшее количество равное примерно 29 тыс. По сравнению с денежными кредитами. 

In [ ]:
tr1 = bar_hor(app_train, "NAME_EDUCATION_TYPE", "Distribution of " ,"#f975ae", w=700, lm=100, return_trace= True)
tr2 = bar_hor(app_train, "NAME_HOUSING_TYPE", "Distribution of " ,"#f975ae", w=700, lm=100, return_trace = True)

fig = tools.make_subplots(rows=1, cols=2, print_grid=False, subplot_titles = ['Типы образования заявителей', 'Тип жилья заявителей' ])
fig.append_trace(tr1, 1, 1);
fig.append_trace(tr2, 1, 2);
fig['layout'].update(height=400, showlegend=False, margin=dict(l=100));
iplot(fig);


tr1, tr2 = gp('NAME_EDUCATION_TYPE', 'Applicants Income Types which repayed the loan')
tr3, tr4 = gp('NAME_HOUSING_TYPE', 'Applicants Income Types which repayed the loan')
fig = tools.make_subplots(rows=1, cols=2, print_grid=False, 
                          subplot_titles = ["Типы образования заявителей, Target=1", "Тип жилья заявителей, Target=1"])
fig.append_trace(tr1, 1, 1);
fig.append_trace(tr3, 1, 2);
fig['layout'].update(height=350, showlegend=False, margin=dict(l=30));
iplot(fig);

> Большое количество заявок (218 тыс.) подано людьми, имеющими среднее образование, за которыми следуют люди с высшим образованием с 75 тыс. Заявители, проживающие в доме / квартире, имеют наибольшее количество кредитных заявок, равное 272 тыс. В то время как мы видим, что заявители с более низким средним образованием имеют самый высокий процент проблем, связанных с оплатой. Кроме того, заявители, проживающие в квартирах или живущие с родителями, также демонстрируют ту же тенденцию.

In [ ]:
tr1 = bar_hor(app_train, "ORGANIZATION_TYPE", "Distribution of " ,"#f975ae", w=700, lm=100, return_trace= True)
tr2 = bar_hor(app_train, "OCCUPATION_TYPE", "Distribution of " ,"#f975ae", w=700, lm=100, return_trace = True)
fig = tools.make_subplots(rows=1, cols=2, print_grid=False, subplot_titles = ['Отрасль работы', 'Занимаемая должность' ])
fig.append_trace(tr1, 1, 1);
fig.append_trace(tr2, 1, 2);
fig['layout'].update(height=600, showlegend=False, margin=dict(l=150));
iplot(fig);

> Лучшие кандидаты, подавшие заявку на получение кредита : рабочие-около 55 тыс., торговый персонал-около 32 тыс., основной персонал-около 28 тыс.

In [ ]:
tr1, tr2 = gp('ORGANIZATION_TYPE', 'Applicants Income Types which repayed the loan')
tr3, tr4 = gp('OCCUPATION_TYPE', 'Applicants Income Types which repayed the loan')
fig = tools.make_subplots(rows=1, cols=2, print_grid=False, 
                          subplot_titles = ["Отрасли работы которые погасили кредит", "Должности которые погасили кредит"])
fig.append_trace(tr1, 1, 1);
fig.append_trace(tr3, 1, 2);
fig['layout'].update(height=350, showlegend=False, margin=dict(l=120));
iplot(fig);

In [ ]:
tr1 = bar_hor(app_train, "FONDKAPREMONT_MODE", "Distribution of FLAG_OWN_REALTY" ,"#639af2", w=700, lm=100, return_trace= True)
tr2 = bar_hor(app_train, "WALLSMATERIAL_MODE", "Distribution of FLAG_OWN_CAR" ,"#a4c5f9", w=700, lm=100, return_trace = True)
tr1 = bar_hor(app_train, "HOUSETYPE_MODE", "Distribution of FLAG_OWN_CAR" ,"#a4c5f9", w=700, lm=100, return_trace = True)

fig = tools.make_subplots(rows=1, cols=2, print_grid=False, subplot_titles = [ 'Тип дома', 'Метериал стен'])
fig.append_trace(tr1, 1, 1);
fig.append_trace(tr2, 1, 2);
# fig.append_trace(tr3, 1, 3);

fig['layout'].update(height=400, showlegend=False, margin=dict(l=100));
iplot(fig);

> "Многоквартирные дома" подали наибольшее количество кредитных заявок, равное примерно 150 тыс., Остальные другие категории: конкретное жилье и террасный дом имеют менее 1500 заявок. Точно так же дома, имеющие панельный и каменно-кирпичный тип стенового материала, подали самые большие заявки, близкие к 120K вместе взятым.

In [ ]:
tr1, tr2 = gp('HOUSETYPE_MODE', 'Applicants Income Types which repayed the loan')
tr3, tr4 = gp('WALLSMATERIAL_MODE', 'Applicants Income Types which repayed the loan')
fig = tools.make_subplots(rows=1, cols=2, print_grid=False, subplot_titles = ["Типы домов погасившие кредит", "Типы стен погасившие кредит"])
fig.append_trace(tr1, 1, 1);
fig.append_trace(tr3, 1, 2);
fig['layout'].update(height=350, showlegend=False, margin=dict(l=120));
iplot(fig);

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Распределение AMT_CREDIT")
ax = sns.distplot(app_train["AMT_CREDIT"])

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Распределение AMT_ANNUITY")
ax = sns.distplot(app_train["AMT_ANNUITY"].dropna())

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Распределение AMT_GOODS_PRICE")
ax = sns.distplot(app_train["AMT_GOODS_PRICE"].dropna())

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Распределение REGION_POPULATION_RELATIVE")
ax = sns.distplot(app_train["REGION_POPULATION_RELATIVE"])

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Распределение DAYS_BIRTH")
ax = sns.distplot(app_train["DAYS_BIRTH"])

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Распределение DAYS_EMPLOYED")
ax = sns.distplot(app_train["DAYS_EMPLOYED"])

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Распределение DAYS_REGISTRATION")
ax = sns.distplot(app_train["DAYS_REGISTRATION"])

In [ ]:
t = app_train["CNT_FAM_MEMBERS"].value_counts()
t1 = pd.DataFrame()
t1['x'] = t.index 
t1['y'] = t.values 

plt.figure(figsize=(12,5));
plt.title("Распределение количества членов семьи заявителя");
ax = sns.barplot(data=t1, x="x", y="y", color="#f975ae");
ax.spines['right'].set_visible(False);
ax.spines['top'].set_visible(False);

ax.set_ylabel('');    
ax.set_xlabel('');

In [ ]:
t = app_train["CNT_CHILDREN"].value_counts()
t1 = pd.DataFrame()
t1['x'] = t.index 
t1['y'] = t.values 

plt.figure(figsize=(12,5));
plt.title("Распределение количества детей у заявителей");
ax = sns.barplot(data=t1, x="x", y="y", color="#f975ae");
ax.spines['right'].set_visible(False);
ax.spines['top'].set_visible(False);

ax.set_ylabel('');    
ax.set_xlabel('');

*Bureau Data*

Все предыдущие кредиты клиента, предоставленные другими финансовыми учреждениями, которые были сообщены в кредитное бюро (для клиентов, имеющих кредит в нашей выборке). Для каждого кредита в нашей выборке существует столько строк, сколько кредитов клиент имел в кредитном бюро до даты подачи заявки.

In [ ]:
bureau = pd.read_csv(path + "bureau.csv")
bureau.head()

*Bureau Balance Data*


Ежемесячные остатки по предыдущим кредитам в кредитном бюро. Эта таблица имеет одну строку для каждого месяца истории каждого предыдущего кредита, сообщенного в кредитное бюро – т. е. таблица имеет (#кредиты в выборке * # относительных предыдущих кредитов * # месяцев, где у нас есть некоторая история, наблюдаемая для предыдущих кредитов).


In [ ]:
bureau_balance = pd.read_csv(path + "bureau_balance.csv")
bureau_balance.head()

*Credit Card Balance Data*


Ежемесячный баланс предыдущих кредитных карт, которые заявитель имеет с Home Credit. Эта таблица имеет одну строку для каждого месяца истории каждого предыдущего кредита в Home Credit, связанные с кредитами в нашей выборке-т. е. таблица имеет (#кредиты в выборке * # относительных предыдущих кредитных карт * # месяцев, где у нас есть некоторая история, наблюдаемая для предыдущей кредитной карты).


In [ ]:
credit_card_balance = pd.read_csv(path + "credit_card_balance.csv")
credit_card_balance.head()

*POS CASH Balance Data*


Ежемесячные снимки баланса предыдущих POS-точек (точек продаж) и кредитов наличными, которые заявитель имел с помощью Home Credit. Эта таблица имеет одну строку для каждого месяца истории каждого предыдущего кредита в Home Credit, связанного с кредитами в нашей выборке – т. е. таблица имеет (#кредиты в выборке * # относительных предыдущих кредитов * # месяцев, в которых у нас есть некоторая история, наблюдаемая для предыдущих кредитов).


In [ ]:
pcb = pd.read_csv(path + "POS_CASH_balance.csv")
pcb.head()

*Previous Application Data*

In [ ]:
previous_application = pd.read_csv(path + "previous_application.csv")
previous_application.head()

In [ ]:
t = previous_application['NAME_CONTRACT_STATUS'].value_counts()
labels = t.index
values = t.values

colors = ['#FEBFB3', '#E1396C', '#96D38C', '#D0F9B1']

trace = go.Pie(labels=labels, values=values,
               hoverinfo='', textinfo='',
               textfont=dict(size=12),
               marker=dict(colors=colors,
                           line=dict(color='#fff', width=2)))

layout = go.Layout(title='Cтатус контракта в предыдущих заявках', height=400)
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

> Большому количество людей (около 62%) были одобрили заявки, в то время как около 19% из них были отменены, а остальные 17% были отказаны.


In [ ]:
t = previous_application['NAME_CLIENT_TYPE'].value_counts()
labels = t.index
values = t.values

colors = ['#FEBFB3', '#E1396C', '#96D38C', '#D0F9B1']

trace = go.Pie(labels=labels, values=values,
               hoverinfo='', textinfo='',
               textfont=dict(size=12),
               marker=dict(colors=colors,
                           line=dict(color='#fff', width=2)))

layout = go.Layout(title='Тип клиента в предыдущих заявках', height=400)
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

> Около 74% предыдущих заявок были поданы повторными клиентами, в то время как только 18% являются новыми.

In [ ]:
t = previous_application['CHANNEL_TYPE'].value_counts()
labels = t.index
values = t.values

colors = ['#FEBFB3', '#E1396C', '#96D38C', '#D0F9B1']

trace = go.Pie(labels=labels, values=values,
               hoverinfo='', textinfo='',
               textfont=dict(size=12),
               marker=dict(colors=colors,
                           line=dict(color='#fff', width=2)))

layout = go.Layout(title='Тип источника в предыдущих запросах', height=400)
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

*Installments Payments Data*

In [ ]:
installments_payments = pd.read_csv(path + "installments_payments.csv")
installments_payments.head()